# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
# Load environment variables
%load_ext dotenv
%dotenv 
# Add src to path
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle
import shap

c:\Users\laksh\miniconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [3]:
X = fires_dt.drop('area', axis=1)
y = fires_dt['area']
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

Features shape: (517, 12)
Target shape: (517,)


In [4]:
categorical_features = ['month', 'day']
numerical_features = [col for col in X.columns if col not in categorical_features]

print(f"\nCategorical features: {categorical_features}")
print(f"Numerical features: {numerical_features}")


Categorical features: ['month', 'day']
Numerical features: ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']


In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
# Preproc 1: Simple preprocessing with scaling and one-hot encoding
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [8]:
# Preproc 2: Advanced preprocessing with power transformation
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('power', PowerTransformer(method='yeo-johnson')), 
            ('scaler', RobustScaler())  
        ]), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [9]:
# Pipeline A = preproc1 + baseline
# Define models
baseline_model = Ridge(random_state=42)
advanced_model = RandomForestRegressor(random_state=42, n_jobs=-1)

# Create four pipelines
pipeline_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', baseline_model)
])

In [10]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge(random_state=42))
])

In [11]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', advanced_model)
])

In [12]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
]) 

In [13]:


pipelines = {
    'Pipeline A (Simple + Ridge)': pipeline_A,
    'Pipeline B (Advanced Preproc + Ridge)': pipeline_B,
    'Pipeline C (Simple + Random Forest)': pipeline_C,
    'Pipeline D (Advanced Preproc + Random Forest)': pipeline_D
}

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [ ]:


# Define parameter grids for each pipeline
param_grids = {
    'Pipeline A (Simple + Ridge)': {
        'regressor__alpha': [0.1, 1.0, 10.0, 100.0]
    },
    'Pipeline B (Advanced Preproc + Ridge)': {
        'regressor__alpha': [0.1, 1.0, 10.0, 100.0]
    },
    'Pipeline C (Simple + Random Forest)': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [5, 10, None]
    },
    'Pipeline D (Advanced Preproc + Random Forest)': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [5, 10, None]
    }
}

In [17]:
# Perform grid search for each pipeline
best_models = {}
results = {}

for name, pipeline in pipelines.items():
    print(f"\nTuning {name}...")
    
    # Grid search with cross-validation
    grid_search = GridSearchCV(
        pipeline, 
        param_grids[name], 
        cv=5, 
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)


Tuning Pipeline A (Simple + Ridge)...


One or more of the test scores are non-finite: [nan nan nan nan]



Tuning Pipeline B (Advanced Preproc + Ridge)...


One or more of the test scores are non-finite: [nan nan nan nan]



Tuning Pipeline C (Simple + Random Forest)...


One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]



Tuning Pipeline D (Advanced Preproc + Random Forest)...


One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]


In [ ]:
# Store best model and results
best_models[name] = grid_search.best_estimator_
results[name] = {
        'best_params': grid_search.best_params_,
        'best_cv_score': -grid_search.best_score_,
        'cv_std': grid_search.cv_results_['std_test_score'][grid_search.best_index_]
    }
    
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV RMSE: {-grid_search.best_score_:.4f}")

Best parameters: {'regressor__max_depth': 5, 'regressor__n_estimators': 50}
Best CV RMSE: nan


# Evaluate

+ Which model has the best performance?

In [22]:
# Evaluate all models on test set
test_results = {}

for name, model in best_models.items():
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    test_results[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }


In [23]:
print("\nTest Set Performance:")
print("-" * 70)
print(f"{'Model':<35} {'RMSE':<10} {'MAE':<10} {'R²':<10}")
print("-" * 70)

for name, metrics in test_results.items():
    print(f"{name:<35} {metrics['RMSE']:<10.4f} {metrics['MAE']:<10.4f} {metrics['R²']:<10.4f}")



Test Set Performance:
----------------------------------------------------------------------
Model                               RMSE       MAE        R²        
----------------------------------------------------------------------
Pipeline D (Advanced Preproc + Random Forest) 109.3390   25.2569    -0.0142   


In [24]:
best_model_name = min(test_results.keys(), key=lambda x: test_results[x]['RMSE'])
best_model = best_models[best_model_name]

print(f"\nBest performing model: {best_model_name}")
print(f"Best RMSE: {test_results[best_model_name]['RMSE']:.4f}")


Best performing model: Pipeline D (Advanced Preproc + Random Forest)
Best RMSE: 109.3390


# Export

+ Save the best performing model to a pickle file.

In [25]:
with open('best_forest_fire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print(f"Best model ({best_model_name}) saved as 'best_forest_fire_model.pkl'")


Best model (Pipeline D (Advanced Preproc + Random Forest)) saved as 'best_forest_fire_model.pkl'


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [30]:
rf_model = None
if 'Random Forest' in best_model_name:
    rf_model = best_model
else:
    # Use the best Random Forest model for SHAP explanation
    rf_candidates = [name for name in best_models.keys() if 'Random Forest' in name]
    if rf_candidates:
        rf_model_name = min(rf_candidates, key=lambda x: test_results[x]['RMSE'])
        rf_model = best_models[rf_model_name]
        print(f"Using {rf_model_name} for SHAP explanation (tree-based model required)")

if rf_model is not None:
    # Transform the data using the preprocessing step
    X_train_transformed = rf_model.named_steps['preprocessing'].transform(X_train)
    X_test_transformed = rf_model.named_steps['preprocessing'].transform(X_test)
    
    # Get feature names after preprocessing
    # Get categorical feature names
    cat_encoder = rf_model.named_steps['preprocessing'].named_transformers_['cat']
    cat_feature_names = cat_encoder.get_feature_names_out(categorical_features)
    
    # Combine all feature names
    all_feature_names = numerical_features + list(cat_feature_names)
    
    print(f"Total features after preprocessing: {len(all_feature_names)}")
    

Total features after preprocessing: 27


*(Answer here.)*

In [ ]:
# Create SHAP explainer
explainer = shap.TreeExplainer(rf_model.named_steps['regressor'])
    
# Calculate SHAP values for entire test set
shap_values = explainer.shap_values(X_test_transformed)
# Feature importance based on mean absolute SHAP values
feature_importance = np.abs(shap_values).mean(0)
feature_ranking = pd.DataFrame({
        'feature': all_feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
print("\nTop 10 Most Important Features (based on SHAP values):")
print(feature_ranking.head(10))
    
print("\nBottom 5 Least Important Features:")
print(feature_ranking.tail(5))

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.